In [ ]:
!pip install pandas==1.1.5 numpy==1.19.4 scipy==1.5.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 4.2 MB/s 
     |████████████████████████████████| 14.5 MB 25.1 MB/s 
     |████████████████████████████████| 25.9 MB 55.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.4 which is incompatible.
tensorf

In [ ]:
!pip install websockets>=9.0


In [ ]:
!pip install alpaca-trade-api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 1.1 MB 6.7 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
     |████████████████████████████████| 299 kB 49.0 MB/s 
     |████████████████████████████████| 271 kB 44.1 MB/s 
     |████████████████████████████████| 144 kB 38.2 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.4
    Uninstalling msgpack-1.0.4:
      Successfully uninstalled msgpack-1.0.4


In [ ]:
import pandas as pd

In [ ]:
import alpaca_trade_api as tradeapi

# API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKZBF19BMWCUWI0SA9E3"
ALPACA_SECRET_KEY = "uX2AdG5eqYzu80wf5PsjbrbvFoKR9NOx2iuDQHfP"

# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, 
                    base_url=BASE_URL, api_version='v2')

In [ ]:
def get_proj_ret(df):
  df['Proj_Returns'] = (3*(d1['Open_Change'].values-1)+1)

  return df

In [ ]:
def get_percent_ret(df):
  df['Open_Change'] = (df['open'].values) / df['open'].iloc[0]

  return df

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit


In [ ]:
def blaghify(date, tick, levtick):
  d1 = api.get_bars(tick, TimeFrame(1, TimeFrameUnit.Minute), date, date, adjustment='raw').df

  if(len(d1) == 0):
    return -1
  d2 = api.get_bars(levtick, TimeFrame(1, TimeFrameUnit.Minute), date, date, adjustment='raw').df

  #get bar data for both instruments

  date2 = pd.to_datetime(date)
  ninethrity = date2.replace(hour=9, minute=30, second=0, microsecond=0)
  four = date2.replace(hour=16, minute=0, second=0, microsecond=0)
  over_d1 = d1[d1.index.isin(d2.index.values)]
  over_d2 = d2[d2.index.isin(d1.index.values)]
  over_d1 = over_d1[over_d1.index.values >= ninethrity]
  over_d1 = over_d1[over_d1.index.values <= four]

  over_d2 = over_d2[over_d2.index.values >= ninethrity]
  over_d2 = over_d2[over_d2.index.values <= four]

  #filter by time and overlap
  d1, d2 = get_percent_ret(over_d1), get_percent_ret(over_d2)
  blagh = (d2['Open_Change'].values)/(3*(d1['Open_Change'].values-1)+1)
  #get theoretical degree of divergence between unlevered and 3x levered instrument
  return blagh, d1, d2

In [ ]:
tick = "SPY"
levtick = "UPRO"

In [ ]:
def running_mean_std(x):
    serx = pd.Series(x)
    mean = list(serx.rolling(10, min_periods=1).mean())
    std = list(serx.rolling(10, min_periods=1).std())
    #mean.insert(0, serx[0])
    #std.insert(0, 0)
    return np.array(mean), np.array(std)

In [ ]:
def run_strategy(date, tick, levtick):

  #initialize vars
  i=0
  netprophet = 0
  netexposure = 0

  wincount = 0
  numarbs = 0

  maxcapital = 0
  blagh, d1, d2 = blaghify(date, tick, levtick)
  runmean, runstd = running_mean_std(blagh)
  divergence = blagh[i] - runmean[i]
  while(i < min(200, len(blagh)-1)):
    if(divergence > 0):
      numarbs+=1
      #implies upro too high
      upro = 0
      spy = 3
      uprice = d2['open'].iloc[i]
      sprice = d1['open'].iloc[i]

      #calculate quantity of upro to buy relative to amount of spy
      upro = spy*sprice/3
      upro = upro/uprice
      upro = -1*int(upro)

      pricepaid = sprice*spy+upro*uprice 
      lever_aj_price = sprice*spy + upro*uprice*3
      netexposure += (sprice*spy + abs(upro*uprice))
      if((abs(sprice*spy) + abs(upro*uprice)) > maxcapital):
        maxcapital = (sprice*spy + abs(upro*uprice))

      while(divergence > 0 and i < len(blagh)-1):
        i+=1
        divergence = blagh[i] - runmean[i]

      #update price once divergence corrected, sell instruments and check profit
      uprice = d2['open'].iloc[i]
      sprice = d1['open'].iloc[i]
      pricesold = sprice*spy + upro*uprice
      netprophet += pricesold-pricepaid
      
      if(pricesold > pricepaid):
        wincount+=1

    if(divergence < 0):
      #same concept but now SPY too high
      numarbs+=1
      #implies spy too high
      upro = 0
      spy = -3
      uprice = d2['open'].iloc[i]
      sprice = d1['open'].iloc[i]

      upro = spy*sprice/3
      upro = upro/uprice
      upro = -1*int(upro)

      pricepaid = sprice*spy+upro*uprice #sprice is cost, upro*uprice is benefit
      netexposure += (abs(sprice*spy) + abs(upro*uprice))
      lever_aj_price = sprice*spy + upro*uprice*3

      if((abs(sprice*spy) + abs(upro*uprice)) > maxcapital):
        maxcapital = (abs(sprice*spy) + abs(upro*uprice))

      while(divergence < 0 and i < len(blagh)-1):
        i+=1
        divergence = blagh[i] - runmean[i]

      uprice = d2['open'].iloc[i]
      sprice = d1['open'].iloc[i]
      pricesold = sprice*spy + upro*uprice
      netprophet += pricesold-pricepaid

      if(pricesold > pricepaid):
        wincount+=1 

    else:
      i+=1
      if(i == len(blagh)):
        break
      divergence = blagh[i] - runmean[i]
      #print(i)
      #print()
  return netprophet, netexposure, wincount/numarbs, maxcapital

  



In [ ]:
run_strategy("2022-05-03", "SPY", "UPRO")

(22.693000000003167, 161363.59890000004, 0.9090909090909091, 1641.2)

In [ ]:
#checks the method on a month of data for a whole lot of leveraged-unleveraged pairs

netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "SPY"
levtick = "UPRO"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 SPY UPRO
25.733599999999115 153607.90580000004 0.8947368421052632 1634.9199999999998
2022-05-03 SPY UPRO
22.693000000003167 161363.59890000004 0.9090909090909091 1641.2
2022-05-04 SPY UPRO
17.847399999997492 167356.7759 0.9705882352941176 1652.03
2022-05-05 SPY UPRO
26.111800000000244 158477.5109 0.9680851063829787 1700.9500000000003
2022-05-06 SPY UPRO
34.848300000001814 160611.16280000005 0.92 1624.9299999999998
2022-05-09 SPY UPRO
43.40159999999946 143546.62140000003 0.9775280898876404 1623.6
2022-05-10 SPY UPRO
31.669799999998645 161736.35250000007 1.0 1620.27
2022-05-11 SPY UPRO
22.259400000002643 133641.20789999998 0.9642857142857143 1610.8799999999999
2022-05-12 SPY UPRO
36.64080000000126 155595.25560000003 0.9705882352941176 1536.2400000000002
2022-05-13 SPY UPRO
29.748599999999897 160304.71560000008 0.9705882352941176 1592.9099999999999
2022-05-16 SPY UPRO
35.97480000000064 179231.28719999993 0.9911504424778761 1597.9791
2022-05-17 SPY UPRO
21.305399999998713 150741

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

1.6256296188355708

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "QQQ"
levtick = "TQQQ"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 QQQ TQQQ
21.909799999998995 131790.49950000003 0.9626168224299065 1241.92
2022-05-03 QQQ TQQQ
18.806199999998398 129692.33729999994 0.9902912621359223 1267.07
2022-05-04 QQQ TQQQ
13.746000000000663 116658.81040000002 0.9347826086956522 1272.27
2022-05-05 QQQ TQQQ
14.15839999999855 120750.69110000004 0.8947368421052632 1303.04
2022-05-06 QQQ TQQQ
20.397599999998306 116815.36480000002 0.9270833333333334 1231.07
2022-05-09 QQQ TQQQ
21.30239999999958 112669.26779999999 0.9468085106382979 1208.1
2022-05-10 QQQ TQQQ
17.277000000000612 107019.22650000005 0.9444444444444444 1198.32
2022-05-11 QQQ TQQQ
13.528200000001107 103818.84090000002 0.9418604651162791 1214.73
2022-05-12 QQQ TQQQ
27.133399999996982 115366.90230000002 0.9603960396039604 1153.07
2022-05-13 QQQ TQQQ
17.172600000000102 103218.34950000004 0.9325842696629213 1165.83
2022-05-16 QQQ TQQQ
17.52840000000117 107701.4874 0.945054945054945 1194.45
2022-05-17 QQQ TQQQ
15.64019999999664 113263.76309999998 0.9468085106382979 1

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

0.01968871503376096

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "IWM"
levtick = "TNA"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 IWM TNA
13.910199999997985 86856.56190000002 0.9322033898305084 748.54
2022-05-03 IWM TNA
16.634799999999416 79436.01770000001 0.8715596330275229 748.54
2022-05-04 IWM TNA
10.193400000000167 75597.96729999999 0.8846153846153846 748.54
2022-05-05 IWM TNA
13.669899999997938 73489.94079999998 0.8217821782178217 750.2552000000001
2022-05-06 IWM TNA
22.602399999998227 75002.70179999998 0.9805825242718447 741.49
2022-05-09 IWM TNA
21.68720000000212 92069.55020000001 0.9318181818181818 708.22
2022-05-10 IWM TNA
19.71639999999934 83345.14319999996 0.8852459016393442 696.88
2022-05-11 IWM TNA
19.69979999999896 84647.44150000002 0.8467741935483871 693.26
2022-05-12 IWM TNA
28.017600000001153 96557.7554 0.8513513513513513 670.44
2022-05-13 IWM TNA
14.344000000000165 87229.148 0.8359375 692.8656
2022-05-16 IWM TNA
13.452800000001218 70338.55720000002 0.8627450980392157 699.3699999999999
2022-05-17 IWM TNA
13.92600000000067 76906.62839999997 0.8899082568807339 710.4549999999999
2022-05-1

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

0.022771621898781486

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "IJH"
levtick = "MIDU"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 IJH MIDU
7.647800000000075 9732.049900000002 1.0 985.4079
2022-05-03 IJH MIDU
9.26540000000034 14810.432400000002 0.9333333333333333 997.91
2022-05-04 IJH MIDU
1.8279999999998608 1999.9830000000002 1.0 1001.86
2022-05-05 IJH MIDU
9.85280000000023 12954.881900000002 1.0 1018.3370000000001
2022-05-06 IJH MIDU
20.209500000000503 20116.8265 0.9047619047619048 972.0899999999999
2022-05-09 IJH MIDU
18.103800000000376 20648.021099999994 0.9090909090909091 961.92
2022-05-10 IJH MIDU
18.22440000000006 14107.804400000003 1.0 965.3580000000001
2022-05-11 IJH MIDU
3.1211999999999875 7613.946199999999 0.875 963.8706
2022-05-12 IJH MIDU
22.13160000000022 14754.855599999999 1.0 940.89
2022-05-13 IJH MIDU
9.901699999999494 15144.220600000002 0.875 966.3599999999999
2022-05-16 IJH MIDU
0.9099999999998545 2856.76 0.3333333333333333 966.87
2022-05-17 IJH MIDU
5.072900000000232 9482.2739 0.9 950.809
2022-05-18 IJH MIDU
5.491999999999962 9447.4835 1.0 948.1500000000001
2022-05-19 IJH MIDU
10.398

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

0.009238151267814198

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "SPHB"
levtick = "HIBL"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 SPHB HIBL
9.683800000000048 8325.7662 0.8529411764705882 247.69
2022-05-03 SPHB HIBL
3.96610000000004 5280.6776 0.8095238095238095 254.015
2022-05-04 SPHB HIBL
1.838400000000064 3769.9724000000006 0.7333333333333333 253.8514
2022-05-05 SPHB HIBL
5.2236000000002605 11941.939499999999 0.7446808510638298 261.455
2022-05-06 SPHB HIBL
18.05640000000011 11395.4012 0.9148936170212766 247.23
2022-05-09 SPHB HIBL
11.523599999999988 12246.3314 0.8490566037735849 236.9
2022-05-10 SPHB HIBL
12.553499999999644 8165.7795000000015 0.8055555555555556 232.5
2022-05-11 SPHB HIBL
16.427299999999775 10467.775599999999 0.782608695652174 230.44
2022-05-12 SPHB HIBL
10.67659999999998 8486.4839 0.8918918918918919 249.23000000000002
2022-05-13 SPHB HIBL
8.939600000000041 9418.049400000002 0.8780487804878049 231.8318
2022-05-16 SPHB HIBL
4.923000000000286 6181.132499999998 0.7777777777777778 232.06000000000003
2022-05-17 SPHB HIBL
5.218800000000215 6765.2842 0.8275862068965517 236.36
2022-05-18 SPHB 

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

3.0443106622805978

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "IJH"
levtick = "UMDD"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 IJH UMDD
7.800200000000018 9844.1905 0.9 994.54
2022-05-03 IJH UMDD
3.161999999999807 5951.752 0.6666666666666666 1002.54
2022-05-04 IJH UMDD
-0.2099999999999227 998.17 0.0 998.17
2022-05-05 IJH UMDD
2.3440000000001646 5005.402 0.8 1020.2800000000001
2022-05-06 IJH UMDD
7.112999999999943 6848.967500000001 1.0 987.9596999999999
2022-05-09 IJH UMDD
7.5622000000000185 8544.057 1.0 960.5581999999999
2022-05-10 IJH UMDD
10.015599999999836 6576.0177 1.0 943.2733000000001
2022-05-11 IJH UMDD
2.997799999999984 4741.5089 0.8 952.1288999999999
2022-05-12 IJH UMDD
6.610799999999983 6518.7234 0.7142857142857143 945.1199999999999
2022-05-13 IJH UMDD
2.929199999999639 6686.4946 0.8571428571428571 960.4499999999999
2022-05-16 IJH UMDD
0.6037000000002877 2849.2474 0.6666666666666666 953.7978999999999
2022-05-17 IJH UMDD
5.6165999999999485 4851.8533 0.8 973.74
2022-05-18 IJH UMDD
6.557799999999247 9632.796100000001 0.9 965.9599999999999
2022-05-19 IJH UMDD
3.9059999999997785 6638.5872 0.8571

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

0.4844928863078949

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "IWM"
levtick = "URTY"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 IWM URTY
40.76519999999914 51232.035 0.9577464788732394 732.1146
2022-05-03 IWM URTY
28.932600000000548 45339.26489999999 0.9193548387096774 738.6942
2022-05-04 IWM URTY
19.324199999999678 41629.00470000001 1.0 743.22
2022-05-05 IWM URTY
28.533600000000547 55639.224 0.9736842105263158 757.47
2022-05-06 IWM URTY
42.240600000000484 55151.042400000006 0.9358974358974359 724.1099999999999
2022-05-09 IWM URTY
33.88260000000014 64929.45659999998 0.9166666666666666 691.1700000000001
2022-05-10 IWM URTY
32.01360000000045 53806.45680000001 0.8271604938271605 690.0400000000001
2022-05-11 IWM URTY
39.300600000000145 52034.561100000006 0.9230769230769231 676.2600000000001
2022-05-12 IWM URTY
34.43019999999973 49696.20809999999 0.972972972972973 690.38
2022-05-13 IWM URTY
24.42840000000075 57033.14399999999 0.9294117647058824 676.47
2022-05-16 IWM URTY
30.12659999999937 47178.480599999995 0.9285714285714286 682.65
2022-05-17 IWM URTY
24.10439999999886 53007.143399999986 0.935064935064935

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

3.884310982411673

In [ ]:
netprophets = []
netexposures = []
winrates = []
maxcapitals = []
blaghs = []
tick = "DIA"
levtick = "UDOW"
for i in range(1, 32):
  date = "2022-05-"+str(i)
  if(i < 10):
    date = "2022-05-0"+str(i)
  #print(i, date, tick, levtick)
  
  if(blaghify(date, tick, levtick) is not -1):

    print(date, tick, levtick)
    netprophet, netexposure, winr, maxcapital = run_strategy(date, tick, levtick)
    print(netprophet, netexposure, winr, maxcapital)

    if(sum([netprophet, netexposure, winr, maxcapital]) != -4):

      netprophets.append(netprophet)
      netexposures.append(netexposure)
      winrates.append(winr)
      maxcapitals.append(maxcapital)

2022-05-02 DIA UDOW
20.041800000000308 111627.94179999997 0.9069767441860465 1311.0296
2022-05-03 DIA UDOW
11.538100000001009 94119.37320000002 0.9444444444444444 1317.6100000000001
2022-05-04 DIA UDOW
12.737799999999538 104590.56209999998 0.9625 1314.9499999999998
2022-05-05 DIA UDOW
12.477200000000721 110970.44660000001 0.9761904761904762 1352.46
2022-05-06 DIA UDOW
15.881600000000503 96257.62720000002 0.8933333333333333 1298.57
2022-05-09 DIA UDOW
19.16259999999977 108857.32750000001 0.872093023255814 1275.43
2022-05-10 DIA UDOW
13.292599999999538 117296.56129999996 0.8064516129032258 1282.3600000000001
2022-05-11 DIA UDOW
35.204199999999105 125915.70289999997 0.86 1273.71
2022-05-12 DIA UDOW
19.234000000001174 118685.20560000004 0.8144329896907216 1235.7200000000003
2022-05-13 DIA UDOW
16.312399999998206 112338.67319999995 0.8888888888888888 1254.96
2022-05-16 DIA UDOW
12.300800000001118 99813.58960000008 0.8375 1256.3600000000001
2022-05-17 DIA UDOW
18.957200000000512 94001.696600

In [ ]:
100*sum(netprophets)/sum(maxcapitals)

1.3372923847788212

In [ ]:
d1 = api.get_quotes("SPY", "2022-05-03", "2022-05-03", limit=10000).df

In [ ]:
d1.head()

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,tape
timestamp,,,,,,,,
2022-05-03 08:00:00.000285+00:00,P,415.8,7,K,415.65,5,[R],B
2022-05-03 08:00:00.012588115+00:00,T,0.0,0,T,409.07,3,[R],B
2022-05-03 08:00:00.012929364+00:00,T,0.0,0,T,409.07,4,[R],B
2022-05-03 08:00:00.028141056+00:00,P,600.0,1,T,409.07,4,[R],B
2022-05-03 08:00:00.028179200+00:00,P,415.8,1,T,409.07,4,[R],B


In [ ]:
d2 = api.get_quotes("UPRO", "2022-05-03", "2022-05-03", limit=10000).df

In [ ]:
d2.head()

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,tape
timestamp,,,,,,,,
2022-05-03 08:00:00.005471744+00:00,P,54.09,2,P,0.00,0,[R],B
2022-05-03 08:00:00.005472256+00:00,P,54.09,2,P,32.89,2,[R],B
2022-05-03 08:00:00.005533952+00:00,P,54.09,2,P,45.03,4,[R],B
2022-05-03 08:00:00.005538048+00:00,P,51.90,1,P,45.03,4,[R],B
2022-05-03 08:00:00.005538560+00:00,P,51.90,1,P,47.96,2,[R],B
